In [ ]:
import sys
import os
import os.path as osp

sys.path.insert(0, "../utils/")
sys.path.insert(0, "../")


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import cv2

In [ ]:
def image_show(image, figsize=(7, 7)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(image, interpolation='nearest')
    plt.tight_layout()
    plt.show()

    
def image_keypoints_show(image_tensor, keypoints, figsize=(7, 7), resize=True):
    fig, ax = plt.subplots(figsize=figsize)
    img = image_tensor.numpy().transpose((1, 2, 0))
    
    if resize:
        img = cv2.resize(img, (64, 64))
    
    ax.imshow(img, interpolation='nearest')

    for i in range(keypoints.shape[1]):
        kp = keypoints[0, i, :]
        if kp[0]>0:
            ax.plot(kp[0], kp[1], marker='o', color="blue", markersize=10)

    plt.tight_layout()
    plt.show()
    
def image_grid_show(image_stack, figsize=(12, 12)):
    fig = plt.figure(figsize=figsize)
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(4, 4),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                    )
    stack_indices = [i for i in range(image_stack.shape[0])]
    for ax, im_idx in zip(grid, stack_indices):
        ax.imshow(image_stack[im_idx])
    plt.show()

# Configuration file parser

In [ ]:
from utils.read_config import yaml_to_object

In [ ]:
config = yaml_to_object("./../config.yaml")

root_dir = osp.abspath(osp.dirname("."))

setattr(config, "root_dir", osp.dirname(root_dir))

# MPIIAnnotationHandler

In [ ]:
from utils.annotation_handler import MPIIAnnotationHandler

In [ ]:
data_path = osp.join(config.root_dir, config.data.MPII.path.base)
training_annotation_file = osp.join(data_path, config.data.MPII.path.annotations.training)
validation_annotation_file = osp.join(data_path, config.data.MPII.path.annotations.validation)
image_dir = osp.join(data_path, config.data.MPII.path.images)

In [ ]:
dh = MPIIAnnotationHandler(training_annotation_file, validation_annotation_file, image_dir)

In [ ]:
image_path, keypoints, center, scale = dh[0]
print(f"image_path: {image_path}")
print(f"center: {center}")
print(f"scale: {scale}\n")
print(f"keypoints:\n{keypoints}")

In [ ]:
train_indices, valid_indices = dh.split_data()
print(f"train_indices.shape: {train_indices.shape}")
print(f"valid_indices.shape: {valid_indices.shape}")

# MPIIDataset

In [ ]:
from utils.dataset import MPIIDataset

In [ ]:
train_dataset = MPIIDataset(
    indices=train_indices,
    mpii_annotation_handle=dh,
    horizontally_flipped_keypoint_ids=config.data.MPII.parts.flipped_ids,
    input_resolution=config.neural_network.train.input_resolution,
    output_resolution=config.neural_network.train.output_resolution,
    num_parts=config.data.MPII.parts.max_count,
    reference_image_size=config.data.MPII.reference_image_size,
    max_rotation_angle=config.neural_network.train.data_augmentation.rotation_angle_max,
    image_scale_factor_range=(config.neural_network.train.data_augmentation.image_scale_factor.min, config.neural_network.train.data_augmentation.image_scale_factor.max),
    image_color_jitter_probability=config.neural_network.train.data_augmentation.image_color_jitter_probability,
    image_horizontal_flip_probability=config.neural_network.train.data_augmentation.image_horizontal_flip_probability,
    hue_max_delta=config.neural_network.train.data_augmentation.hue_max_delta,
    saturation_min_delta=config.neural_network.train.data_augmentation.saturation_min_delta,
    brightness_max_delta=config.neural_network.train.data_augmentation.brightness_max_delta,
    contrast_min_delta=config.neural_network.train.data_augmentation.contrast_min_delta,
)

valid_dataset = MPIIDataset(
    indices=valid_indices,
    mpii_annotation_handle=dh,
    horizontally_flipped_keypoint_ids=config.data.MPII.parts.flipped_ids,
    input_resolution=config.neural_network.train.input_resolution,
    output_resolution=config.neural_network.train.output_resolution,
    num_parts=config.data.MPII.parts.max_count,
    reference_image_size=config.data.MPII.reference_image_size,
    max_rotation_angle=config.neural_network.train.data_augmentation.rotation_angle_max,
    image_scale_factor_range=(config.neural_network.train.data_augmentation.image_scale_factor.min, config.neural_network.train.data_augmentation.image_scale_factor.max),
    image_color_jitter_probability=config.neural_network.train.data_augmentation.image_color_jitter_probability,
    image_horizontal_flip_probability=config.neural_network.train.data_augmentation.image_horizontal_flip_probability,
    hue_max_delta=config.neural_network.train.data_augmentation.hue_max_delta,
    saturation_min_delta=config.neural_network.train.data_augmentation.saturation_min_delta,
    brightness_max_delta=config.neural_network.train.data_augmentation.brightness_max_delta,
    contrast_min_delta=config.neural_network.train.data_augmentation.contrast_min_delta
)


In [ ]:
idxs = np.random.randint(0, 6000, size=(100,))
for idx in idxs:
    image_tensor, heatmap, in_kps, out_kps = train_dataset[idx]
    
    print(f"image_tensor.shape: {image_tensor.shape}")
    print(f"heatmap.shape: {heatmap.shape}")
    
    image_keypoints_show(image_tensor, out_kps, figsize=(3, 3))
    image_show(torch.sum(heatmap, axis=0).numpy(), figsize=(3, 3))
